# Phân Tích Khám Phá Dữ Liệu - Dự Đoán Giá Nhà tại Việt Nam

Notebook này thực hiện phân tích khám phá dữ liệu (EDA) để hiểu về các yếu tố ảnh hưởng đến giá nhà tại các khu vực của thành phố Hồ Chí Minh. Phân tích này sẽ giúp chúng ta hiểu rõ hơn về bộ dữ liệu trước khi áp dụng các mô hình học máy.

## 1. Import Các Thư Viện Cần Thiết

Import các thư viện Python để phân tích và trực quan hóa dữ liệu.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cấu hình hiển thị
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
sns.set_palette('viridis')

# Cấu hình hiển thị tiếng Việt
plt.rcParams['font.family'] = 'DejaVu Sans'

# Ẩn các cảnh báo
import warnings
warnings.filterwarnings('ignore')

## 2. Đọc và Khám Phá Dữ Liệu

Đọc bộ dữ liệu nhà đất và khám phá cấu trúc cơ bản của nó.

In [ ]:
# Đọc dữ liệu từ file CSV
data_path = 'models_ml/sample_data.csv'
df = pd.read_csv(data_path)

# Hiển thị 5 dòng đầu tiên
print("5 dòng đầu tiên của bộ dữ liệu:")
df.head()

In [ ]:
# Kiểm tra thông tin về bộ dữ liệu
print("\nThông tin về bộ dữ liệu:")
df.info()

# Thống kê mô tả cho các cột số
print("\nThống kê mô tả cho các cột số:")
df.describe()

In [ ]:
# Kiểm tra giá trị thiếu
print(f"Số lượng giá trị thiếu trong mỗi cột:\n{df.isnull().sum()}")

# Kiểm tra giá trị trùng lặp
duplicated_rows = df.duplicated().sum()
print(f"\nSố lượng dòng bị trùng lặp: {duplicated_rows}")

## 3. Phân Tích Biến Phân Loại

Phân tích các biến phân loại như vị trí (location) và loại nhà (house_type), cũng như các tiện ích nhà.

In [ ]:
# Phân tích vị trí (location)
plt.figure(figsize=(12, 6))
sns.countplot(x='location', data=df, order=df['location'].value_counts().index)
plt.title('Phân Bố Nhà theo Vị Trí')
plt.xlabel('Vị Trí')
plt.ylabel('Số lượng')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Giá trung bình theo vị trí
location_price = df.groupby('location')['price'].agg(['mean', 'count']).sort_values(by='mean', ascending=False)
location_price.columns = ['Giá Trung Bình', 'Số Lượng']
location_price['Giá Trung Bình'] = location_price['Giá Trung Bình'].round(2)

print("\nGiá trung bình theo vị trí:")
print(location_price)

In [ ]:
# Phân tích loại nhà (house_type)
plt.figure(figsize=(12, 6))
sns.countplot(x='house_type', data=df, order=df['house_type'].value_counts().index)
plt.title('Phân Bố theo Loại Nhà')
plt.xlabel('Loại Nhà')
plt.ylabel('Số lượng')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Giá trung bình theo loại nhà
type_price = df.groupby('house_type')['price'].agg(['mean', 'count']).sort_values(by='mean', ascending=False)
type_price.columns = ['Giá Trung Bình', 'Số Lượng']
type_price['Giá Trung Bình'] = type_price['Giá Trung Bình'].round(2)

print("\nGiá trung bình theo loại nhà:")
print(type_price)

In [ ]:
# Phân tích các tiện ích nhà (has_parking, has_elevator, has_pool, is_furnished)
amenities = ['has_parking', 'has_elevator', 'has_pool', 'is_furnished']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, amenity in enumerate(amenities):
    sns.countplot(x=amenity, data=df, ax=axes[i])
    axes[i].set_title(f'Phân Bố {amenity.replace("has_", "").replace("is_", "").title()}')
    axes[i].set_ylabel('Số lượng')
    
    # Thêm tỉ lệ phần trăm
    total = len(df[amenity])
    for p in axes[i].patches:
        height = p.get_height()
        axes[i].text(p.get_x() + p.get_width()/2., height + 0.5,
                '{:1.1f}%'.format(height/total*100),
                ha="center")

plt.tight_layout()
plt.show()

# Phân tích giá trung bình cho từng tiện ích
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, amenity in enumerate(amenities):
    sns.barplot(x=amenity, y='price', data=df, ax=axes[i])
    axes[i].set_title(f'Giá Trung Bình theo {amenity.replace("has_", "").replace("is_", "").title()}')
    axes[i].set_ylabel('Giá Trung Bình')
    
    # Thêm giá trị trung bình
    for p in axes[i].patches:
        axes[i].text(p.get_x() + p.get_width()/2., p.get_height() + 1000,
                '{:.1f}'.format(p.get_height()),
                ha="center")

plt.tight_layout()
plt.show()

## 4. Phân Tích Biến Số

Phân tích các biến số như diện tích, số phòng ngủ, số phòng tắm và năm xây dựng.

In [ ]:
# Phân tích diện tích (area)
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['area'], bins=20, kde=True)
plt.title('Phân Bố Diện Tích')
plt.xlabel('Diện Tích (m²)')
plt.ylabel('Số lượng')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y='area', data=df)
plt.title('Boxplot Diện Tích')
plt.ylabel('Diện Tích (m²)')

plt.tight_layout()
plt.show()

In [ ]:
# Phân tích số phòng ngủ và phòng tắm
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Số phòng ngủ
sns.countplot(x='num_bedrooms', data=df, ax=axes[0])
axes[0].set_title('Phân Bố Số Phòng Ngủ')
axes[0].set_xlabel('Số Phòng Ngủ')
axes[0].set_ylabel('Số lượng')

# Số phòng tắm
sns.countplot(x='num_bathrooms', data=df, ax=axes[1])
axes[1].set_title('Phân Bố Số Phòng Tắm')
axes[1].set_xlabel('Số Phòng Tắm')
axes[1].set_ylabel('Số lượng')

plt.tight_layout()
plt.show()

# Giá trung bình theo số phòng ngủ và phòng tắm
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Giá theo số phòng ngủ
sns.barplot(x='num_bedrooms', y='price', data=df, ax=axes[0])
axes[0].set_title('Giá Trung Bình theo Số Phòng Ngủ')
axes[0].set_xlabel('Số Phòng Ngủ')
axes[0].set_ylabel('Giá Trung Bình')

# Giá theo số phòng tắm
sns.barplot(x='num_bathrooms', y='price', data=df, ax=axes[1])
axes[1].set_title('Giá Trung Bình theo Số Phòng Tắm')
axes[1].set_xlabel('Số Phòng Tắm')
axes[1].set_ylabel('Giá Trung Bình')

plt.tight_layout()
plt.show()

In [ ]:
# Phân tích năm xây dựng (year_built)
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['year_built'], bins=15, kde=True)
plt.title('Phân Bố Năm Xây Dựng')
plt.xlabel('Năm Xây Dựng')
plt.ylabel('Số lượng')

# Scatter plot giữa năm xây dựng và giá
plt.subplot(1, 2, 2)
sns.scatterplot(x='year_built', y='price', data=df, alpha=0.6)
plt.title('Mối Quan Hệ Giữa Năm Xây Dựng và Giá')
plt.xlabel('Năm Xây Dựng')
plt.ylabel('Giá')

plt.tight_layout()
plt.show()

# Phân nhóm theo thập niên
df['decade'] = (df['year_built'] // 10) * 10
plt.figure(figsize=(12, 6))
sns.boxplot(x='decade', y='price', data=df)
plt.title('Phân Phối Giá theo Thập Niên Xây Dựng')
plt.xlabel('Thập Niên')
plt.ylabel('Giá')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Phân Tích Mối Quan Hệ

Phân tích mối quan hệ giữa các đặc điểm và giá nhà.

In [ ]:
# Tạo một DataFrame mới với các biến số đã chuyển đổi
df_numeric = df.copy()

# Chuyển đổi biến phân loại thành số
for col in ['location', 'house_type']:
    le = LabelEncoder()
    df_numeric[col] = le.fit_transform(df[col])

# Tính ma trận tương quan
corr_matrix = df_numeric.corr()

# Vẽ heatmap tương quan
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Ma Trận Tương Quan giữa Các Biến')
plt.tight_layout()
plt.show()

# Lọc các biến có tương quan cao với giá
price_corr = corr_matrix['price'].sort_values(ascending=False)
print("\nTương quan của các biến với giá nhà:")
print(price_corr)

In [ ]:
# Phân tích mối quan hệ giữa diện tích và giá
plt.figure(figsize=(12, 8))

# Scatter plot với hue theo vị trí
sns.scatterplot(x='area', y='price', hue='location', data=df, alpha=0.7)
plt.title('Mối Quan Hệ Giữa Diện Tích và Giá theo Vị Trí')
plt.xlabel('Diện Tích (m²)')
plt.ylabel('Giá')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Scatter plot với hue theo loại nhà
plt.figure(figsize=(12, 8))
sns.scatterplot(x='area', y='price', hue='house_type', data=df, alpha=0.7)
plt.title('Mối Quan Hệ Giữa Diện Tích và Giá theo Loại Nhà')
plt.xlabel('Diện Tích (m²)')
plt.ylabel('Giá')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Tính giá trên một đơn vị diện tích (m²)
df['price_per_sqm'] = df['price'] / df['area']

# Phân tích giá/m² theo vị trí
plt.figure(figsize=(12, 8))
sns.boxplot(x='location', y='price_per_sqm', data=df)
plt.title('Giá/m² theo Vị Trí')
plt.xlabel('Vị Trí')
plt.ylabel('Giá/m²')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Phân tích giá/m² theo loại nhà
plt.figure(figsize=(12, 8))
sns.boxplot(x='house_type', y='price_per_sqm', data=df)
plt.title('Giá/m² theo Loại Nhà')
plt.xlabel('Loại Nhà')
plt.ylabel('Giá/m²')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Mô Hình Hóa Dự Đoán Giá

Đánh giá các mô hình đã được huấn luyện để dự đoán giá nhà.

In [ ]:
# Chuẩn bị dữ liệu cho mô hình
df_model = df.drop(['price_per_sqm', 'decade'], axis=1)  # Loại bỏ các cột đã tạo thêm

# Xác định các cột liên tục (numeric) và phân loại (object)
continuous_cols = df_model.select_dtypes(include=['float64', 'int64']).columns.drop('price', errors='ignore')
categorical_cols = df_model.select_dtypes(include=['object']).columns

# Chuẩn hóa các cột liên tục
scaler = MinMaxScaler()
df_model[continuous_cols] = scaler.fit_transform(df_model[continuous_cols])

# Label encode các cột phân loại
for col in categorical_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])

# Chia dữ liệu
X = df_model.drop('price', axis=1)
y = df_model['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")

In [ ]:
# Thử tải các mô hình đã huấn luyện (nếu có)
try:
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import joblib
    
    # Tải các mô hình
    linear_reg = joblib.load('models_ml/linear_regression_model.joblib')
    rf_model = joblib.load('models_ml/random_forest_model.joblib')
    xgb_model = joblib.load('models_ml/xgboost_model.joblib')
    
    # Dự đoán bằng mô hình Linear Regression
    lr_pred = linear_reg.predict(X_test)
    lr_mse = mean_squared_error(y_test, lr_pred)
    lr_mae = mean_absolute_error(y_test, lr_pred)
    lr_r2 = r2_score(y_test, lr_pred)
    
    # Dự đoán bằng mô hình Random Forest
    rf_pred = rf_model.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_pred)
    rf_mae = mean_absolute_error(y_test, rf_pred)
    rf_r2 = r2_score(y_test, rf_pred)
    
    # Dự đoán bằng mô hình XGBoost
    xgb_pred = xgb_model.predict(X_test)
    xgb_mse = mean_squared_error(y_test, xgb_pred)
    xgb_mae = mean_absolute_error(y_test, xgb_pred)
    xgb_r2 = r2_score(y_test, xgb_pred)
    
    # So sánh các mô hình
    models = ['Linear Regression', 'Random Forest', 'XGBoost']
    mse_scores = [lr_mse, rf_mse, xgb_mse]
    mae_scores = [lr_mae, rf_mae, xgb_mae]
    r2_scores = [lr_r2, rf_r2, xgb_r2]
    
    # Tạo DataFrame cho kết quả
    results = pd.DataFrame({
        'Model': models,
        'MSE': mse_scores,
        'MAE': mae_scores,
        'R²': r2_scores
    })
    
    print("\nKết quả đánh giá các mô hình:")
    print(results)
    
    # Vẽ biểu đồ so sánh
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # So sánh MSE
    axes[0].bar(models, mse_scores, color='skyblue')
    axes[0].set_title('So Sánh MSE')
    axes[0].set_ylabel('Mean Squared Error')
    axes[0].tick_params(axis='x', rotation=45)
    
    # So sánh MAE
    axes[1].bar(models, mae_scores, color='lightgreen')
    axes[1].set_title('So Sánh MAE')
    axes[1].set_ylabel('Mean Absolute Error')
    axes[1].tick_params(axis='x', rotation=45)
    
    # So sánh R²
    axes[2].bar(models, r2_scores, color='salmon')
    axes[2].set_title('So Sánh R²')
    axes[2].set_ylabel('R-squared')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
except FileNotFoundError:
    print("Không tìm thấy các file mô hình đã huấn luyện.")
except Exception as e:
    print(f"Có lỗi khi tải mô hình: {str(e)}")

In [ ]:
# Trực quan hóa dự đoán so với giá trị thực
try:
    # Tạo DataFrame cho kết quả dự đoán
    pred_df = pd.DataFrame({
        'Actual': y_test,
        'Linear Regression': lr_pred,
        'Random Forest': rf_pred,
        'XGBoost': xgb_pred
    })
    
    # Lấy một mẫu ngẫu nhiên để vẽ
    sample_size = min(50, len(pred_df))
    sample_pred = pred_df.sample(n=sample_size, random_state=42)
    
    # Vẽ biểu đồ so sánh
    plt.figure(figsize=(14, 8))
    
    # Lấy chỉ số của mẫu
    indices = range(len(sample_pred))
    
    # Vẽ đường giá trị thực
    plt.plot(indices, sample_pred['Actual'], marker='o', linestyle='-', linewidth=2, label='Giá Thực Tế')
    
    # Vẽ đường dự đoán
    plt.plot(indices, sample_pred['Linear Regression'], marker='s', linestyle='--', linewidth=1, label='Linear Regression')
    plt.plot(indices, sample_pred['Random Forest'], marker='^', linestyle='--', linewidth=1, label='Random Forest')
    plt.plot(indices, sample_pred['XGBoost'], marker='d', linestyle='--', linewidth=1, label='XGBoost')
    
    plt.title('So Sánh Giá Thực Tế và Dự Đoán')
    plt.xlabel('Chỉ số mẫu')
    plt.ylabel('Giá')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Biểu đồ tương quan giữa giá thực tế và dự đoán
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].scatter(y_test, lr_pred, alpha=0.5)
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    axes[0].set_title('Linear Regression: Giá Thực Tế vs Dự Đoán')
    axes[0].set_xlabel('Giá Thực Tế')
    axes[0].set_ylabel('Giá Dự Đoán')
    
    axes[1].scatter(y_test, rf_pred, alpha=0.5)
    axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    axes[1].set_title('Random Forest: Giá Thực Tế vs Dự Đoán')
    axes[1].set_xlabel('Giá Thực Tế')
    axes[1].set_ylabel('Giá Dự Đoán')
    
    axes[2].scatter(y_test, xgb_pred, alpha=0.5)
    axes[2].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    axes[2].set_title('XGBoost: Giá Thực Tế vs Dự Đoán')
    axes[2].set_xlabel('Giá Thực Tế')
    axes[2].set_ylabel('Giá Dự Đoán')
    
    plt.tight_layout()
    plt.show()
    
    # Histogram của sai số dự đoán
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].hist(y_test - lr_pred, bins=20, alpha=0.7)
    axes[0].set_title('Linear Regression: Histogram Sai Số')
    axes[0].set_xlabel('Sai Số (Thực Tế - Dự Đoán)')
    axes[0].set_ylabel('Tần suất')
    
    axes[1].hist(y_test - rf_pred, bins=20, alpha=0.7)
    axes[1].set_title('Random Forest: Histogram Sai Số')
    axes[1].set_xlabel('Sai Số (Thực Tế - Dự Đoán)')
    axes[1].set_ylabel('Tần suất')
    
    axes[2].hist(y_test - xgb_pred, bins=20, alpha=0.7)
    axes[2].set_title('XGBoost: Histogram Sai Số')
    axes[2].set_xlabel('Sai Số (Thực Tế - Dự Đoán)')
    axes[2].set_ylabel('Tần suất')
    
    plt.tight_layout()
    plt.show()
    
except NameError:
    print("Biến mô hình chưa được định nghĩa. Hãy chạy phần đánh giá mô hình trước.")
except Exception as e:
    print(f"Có lỗi khi trực quan hóa dự đoán: {str(e)}")

## 7. Tầm Quan Trọng của Các Đặc Điểm

Xác định tầm quan trọng của các đặc điểm trong việc dự đoán giá nhà.

In [ ]:
# Phân tích tầm quan trọng đặc điểm từ mô hình Random Forest
try:
    # Lấy tầm quan trọng của đặc điểm từ Random Forest
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTầm quan trọng của các đặc điểm từ mô hình Random Forest:")
    print(feature_importance)
    
    # Vẽ biểu đồ tầm quan trọng
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Tầm Quan Trọng của Các Đặc Điểm trong Mô Hình Random Forest')
    plt.xlabel('Điểm Quan Trọng')
    plt.tight_layout()
    plt.show()
    
except NameError:
    print("Biến mô hình Random Forest chưa được định nghĩa. Hãy chạy phần đánh giá mô hình trước.")
except AttributeError:
    print("Mô hình không có thuộc tính feature_importances_.")
except Exception as e:
    print(f"Có lỗi khi phân tích tầm quan trọng đặc điểm: {str(e)}")

In [ ]:
# Phân tích tầm quan trọng đặc điểm từ mô hình XGBoost
try:
    # Lấy tầm quan trọng của đặc điểm từ XGBoost
    xgb_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': xgb_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTầm quan trọng của các đặc điểm từ mô hình XGBoost:")
    print(xgb_importance)
    
    # Vẽ biểu đồ tầm quan trọng
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=xgb_importance)
    plt.title('Tầm Quan Trọng của Các Đặc Điểm trong Mô Hình XGBoost')
    plt.xlabel('Điểm Quan Trọng')
    plt.tight_layout()
    plt.show()
    
except NameError:
    print("Biến mô hình XGBoost chưa được định nghĩa. Hãy chạy phần đánh giá mô hình trước.")
except AttributeError:
    print("Mô hình không có thuộc tính feature_importances_.")
except Exception as e:
    print(f"Có lỗi khi phân tích tầm quan trọng đặc điểm: {str(e)}")

## 8. Kết Luận và Đề Xuất

Dựa trên phân tích dữ liệu và kết quả từ các mô hình, chúng ta có thể đưa ra một số kết luận và đề xuất:

### Kết Luận Chính

1. **Diện tích là yếu tố quan trọng nhất** ảnh hưởng đến giá nhà, có tương quan mạnh mẽ với giá.

2. **Vị trí (location)** có tác động lớn đến giá, với các khu vực như Quận 1, 2, 3 có giá trung bình cao hơn đáng kể.

3. **Loại nhà** cũng ảnh hưởng rõ rệt đến giá, với biệt thự và nhà mặt tiền thường có giá cao hơn.

4. **Các tiện ích như bãi đỗ xe, thang máy, hồ bơi** góp phần làm tăng giá trị của căn nhà, tuy nhiên mức độ ảnh hưởng khác nhau.

5. **Số lượng phòng ngủ và phòng tắm** có tương quan tích cực với giá, nhưng không mạnh bằng diện tích.

6. **Năm xây dựng** có tác động ít hơn so với các yếu tố khác, nhưng vẫn có một số mối liên hệ với giá.

### Đề Xuất

1. **Sử dụng mô hình tốt nhất cho dự đoán**: Các mô hình học máy như Random Forest và XGBoost cho kết quả tốt hơn so với Linear Regression, đặc biệt là XGBoost với độ chính xác cao nhất.

2. **Cải thiện dữ liệu**: Thu thập thêm dữ liệu về các đặc điểm quan trọng khác như chất lượng xây dựng, khoảng cách đến các tiện ích công cộng, trường học, và bệnh viện.

3. **Cân nhắc phân khúc thị trường**: Phát triển các mô hình riêng cho từng loại nhà hoặc khu vực có thể cải thiện độ chính xác dự đoán.

4. **Theo dõi biến động giá theo thời gian**: Cập nhật dữ liệu và mô hình định kỳ để phản ánh xu hướng thị trường bất động sản.

5. **Xây dựng công cụ dự đoán**: Phát triển một giao diện đơn giản cho người dùng nhập các đặc điểm và nhận dự đoán giá từ mô hình XGBoost hoặc ensemble (kết hợp) của nhiều mô hình.

### Bước Tiếp Theo

1. Triển khai mô hình trong môi trường sản phẩm
2. Thu thập phản hồi từ người dùng và cải thiện mô hình
3. Mở rộng phạm vi dự đoán sang các khu vực khác